# Clustering Functions 

## In this notebook there are a series of useful R functions to perform clustering
### As follows: 
#### 1. Basic NMF - (AutoNMF), negative matrix values are set the the minimum, cluster is determined by the meax cophentic coef.  
#### 2. Basic NMF (2.0) - (AutoNMF2), with preselected top 2000 variable genes
#### 3. AutoNMF_DE -  AutoNMF with 'EdgerR' Differentail Expression, and GWAS  
#### 4. Monti-Carlo Consensus Based Clustering (M3C) - (AutoM3C), this method allows rejection of the NULL Hypothesis that K= 1, it also provides useful info on the stability of clusters 
#### 5. AutoM3C_DE - with 'EdgerR' Differentail Expression, and GWAS  

###### Important Notes: 
###### - Input 'x' is the expression matrix with gene expression features (gene names/ensemble IDs) in rownames, Samples as columns (Sample Names as colnames)
###### - 32GB RAM systems may throw an error depending on data set size because of R's, look into unlimit this or use 64GB RAM system
###### - For publication/important puposes I recomend doing both NMF and M3C, as M3C is quite novel 

In [1]:
            ## Librarys ## 
#Installation# 
## try http:// if https:// URLs are not supported
#source("https://bioc.ism.ac.jp/biocLite.R")
#biocLite("M3C")
#biocLite("NMF")
#biocLite("gplots")
#biocLite("ggsci")
#install.packages ("do.parallel")-- This will allow for parallel processing with the NMF funcion- *nix/Mac only 
library(M3C) # M3C Consensus Based Clustering 
library(NMF) # For NMF and the heatmap  and nneg functions function 
library(gplots) # has a nice colour scale
library(ggsci) # more cool colours

Loading required package: pkgmaker
Loading required package: registry
Loading required package: rngtools
Loading required package: cluster
NMF - BioConductor layer [OK] | Shared memory capabilities [OK] | Cores 7/8

Attaching package: ‘gplots’

The following object is masked from ‘package:stats’:

    lowess



#### 1. Basic NMF - AutoNMF, negative matrix values are set the the minimum, cluster is determined by the meax cophentic coef.  


In [ ]:

AutoNMF <- function(x){
   x_pos <- nneg(as.matrix(x), method=c("min")) 
    NMF_Estimate <- nmfEstimateRank(x_pos,2:10, nrun=100,) #can add in .options="P"- this enables parallel processing
    #rX <- randomize(x_Pos) # For randomised NMF 
    #rand_ Estiamate  <- nmfEstimateRank(rX, 2:10, nrun=100) 
    rw <- which.max(NMF_Estimate$measures$cophenetic) # This will automatically determine optimal rank based on max copenhentic coefictient
Rank <- NMF_Estimate$measures$rank[rw]
res <- nmf(x_pos, Rank, nrun = 1000) 
    return(res)
}



In [ ]:
 #plot(NMF_Estimate, rand_Estimate) # Make sure the clusters aren't random
#plot(res) # NMF plots 
    #plot(silhouette(res)) ## This tells you how stable samples are in the clusters 
    

#### 2. Basic NMF (2.0) - (AutoNMF2), with preselected top 2000 variable genes


In [ ]:
AutoNMF2 <- function(x){
    var_genes <- apply(x, 1, var)

# Get the gene names for the top 2000 most variable genes
select_var <- names(sort(var_genes, decreasing=TRUE))[1:2000]


# Subset 20000 matrix
highly_variable_genes <- x[select_var,]
rownames(highly_variable_genes) <- select_var
    
    ## Make matrix possitive ## 
    x_pos <- as.data.frame(nneg(highly_variable_genes, method=c("min")))
    
    ##Estimate the NMF Rank ##
    
    NMF_Estimate <- nmfEstimateRank(x_pos,2:10, nrun=100,) #can add in .options="P"- this enables parallel processing
    #rX <- randomize(x_Pos) # For randomised NMF 
    #rand_ Estiamate  <- nmfEstimateRank(rX, 2:10, nrun=100)
    rw <- which.max(NMF_Estimate$measures$cophenetic) # This will automatically determine optimal rank based on max copenhentic coefictient
Rank <- x_NMF_Estimate$measures$rank[rw]
res <- nmf(x_pos, rank=Rank, nrun = 100)
    ## data_list('rand', rand_Estiamate, 'res', res)
    ## retrun(data_list)
   }

In [ ]:
#plot(NMF_Estimate, rand_Estimate) # Make sure the clusters aren't random 
###plot(res)
    ##plot(silhouette(res)) ## This tells you how stable the clusters are 


#### 4. Monti-Carlo Consensus Based Clustering (M3C) - (AutoM3C), this method allows rejection of the NULL Hypothesis that K= 1, it also provides useful info on the stability of clusters 


In [ ]:
AutoM3C <- function(x) { 
    x_pos <- nneg(as.matrix(exps_data), method=c("min"))
res <- M3C(x_pos, cores=4, seed = 1234, des = NULL) ##Double check cores, 4 is standard##
   rw <- which.max(res$scores$PAC_REAL)
   Rank <- res$scores$K[rw]
    data <- res$realdataresults[[Rank]]$ordered_data # this is the data
annon <- res$realdataresults[[Rank]]$ordered_annotation # this is the annotation
ccmatrix <- res$realdataresults[[Rank]]$consensus_matrix # this is the consensus matrix
    data_list <- list('data'= data, 'annon' = annon, 'ccmatrix'=  ccmatrix)
    return(data_list)
} 
 ## Plots ## 

# normalise and scale the data
#data <- t(scale(t(P$data))) # z-score normalise each row (feature)
#data <- apply(data, 2, function(x) ifelse(x > 4, 4, x)) # compress data within range
#data <- apply(data, 2, function(x) ifelse(x < -4, -4, x)) # compress data within range

# get some cool colour palettes from the ggsci package and RColourBrewer
#ann_colors <- ggsci::pal_startrek("uniform")(4) # star trek palette
#ann_colors2 <- ggsci::pal_futurama()(4) # futurama palette
#pal <- rev(colorRampPalette(RColorBrewer::brewer.pal(10, "RdBu"))(256))
# NMF::aheatmap(data, annCol = P$annon, Colv = NA, distfun = 'pearson', 
         #color = gplots::bluered(256), annColors = list(class=ann_colors, consensuscluster=ann_colors2))


#n <- 10
#seq = rev(seq(0,255,by=255/(n)))
#palRGB = cbind(seq,seq,255)
#mypal <-rgb(palRGB,maxColorValue=255)
# plot consensus matrix heatmap, do not cluster rows and columns
#NMF::aheatmap(ccmatrix, 
              #annCol = annon[,1,drop=FALSE], 
              #color = mypal, scale = 'none', cexRow = 0, cexCol = 0,
              #Colv=NA,Rowv=NA,annColors = list(consensuscluster=ann_colors2))